<a href="https://colab.research.google.com/github/sheikhjubaer/BN_HTR/blob/main/Code%20for%20Automatic%20Annotation/Automatic_Line_and_Word_Annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Defining functions, importing Libraries, deleting directory contents**

##Mounting Drive...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importing necessary libraries...

In [ ]:
import os
from google.colab import files
from os.path import join
import numpy as np
import operator
import cv2
import fnmatch
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import glob
import imutils
from math import *
from scipy.stats import mode
from distutils.dir_util import copy_tree
from IPython.display import Image
from google.colab.patches import cv2_imshow
import PIL.Image
import os.path
import shutil

##Removing previous files and folders from their directories in New run...

In [ ]:
# Removing previous files and folders from their directories....
x = os.listdir("/content")
for i in x:
  if i.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif', 'JPG', 'JPEG', 'PNG')) == True:
    img_label = i.split('.')[0]
    final_path = "/content/"+img_label
    shutil.rmtree(final_path)
    temp = "/content/"+i
    os.remove(temp)

path1 = "/content/DSkew"
if os.path.exists(path1) == True:
  shutil.rmtree(path1)
path2 = "/content/HaughLine_Affine"
if os.path.exists(path2) == True:
  shutil.rmtree(path2)
path3 = "/content/Rotated_line_by_HaughLine_Affine"
if os.path.exists(path3) == True:
  shutil.rmtree(path3)
path4 = "/content/final_line_segmentation"
if os.path.exists(path4) == True:
  shutil.rmtree(path4)
path5 = "/content/final_word_segmentation"
if os.path.exists(path5) == True:
  shutil.rmtree(path5)
path6 = "/content/initial_line_segmantation"
if os.path.exists(path6) == True:
  shutil.rmtree(path6)
path7 = "/content/sorted_Word_detection"
if os.path.exists(path7) == True:
  shutil.rmtree(path7)
path8 = "/content/sorted_line_after_1st_detection"
if os.path.exists(path8) == True:
  shutil.rmtree(path8)
path9 = "/content/yolov5/runs"
if os.path.exists(path9) == True:
  shutil.rmtree(path9)
path11 = "/content/2nd line detection for rotated images"
if os.path.exists(path11) == True:
  shutil.rmtree(path11)
path12 = "/content/2nd line detection for rotated images (labels)"
if os.path.exists(path12) == True:
  shutil.rmtree(path12)
path13 = "/content/Original line images"
if os.path.exists(path13) == True:
  shutil.rmtree(path13)
  

##YoloV5 Setup...

In [ ]:
path = "/content/yolov5"
if os.path.exists(path) == False:
  !git clone https://github.com/ultralytics/yolov5  # clone repo
  %cd yolov5
  %pip install -qr requirements.txt  # install dependencies

  import torch
  from IPython.display import Image, clear_output  # to display images

  clear_output()
  print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

  %cd /content/
else:
  print("Yolo requirements are already exists!")
  

Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
/content


##Unzipping files...

Defining a function to copy the dataset into colab local directory (for faster execution).

In [ ]:
def file_unzip(zip_file):
  !unzip $zip_file -d "/content"

##Sort line name of given list...

Defining a function to sort a list containing the name of images to visualize them serially. 

In [ ]:
def line_sort(lines):
    sort_lines = {}
    for line in lines:
        img_lb = line.split('.')[0]
        lb = [int(i) for i in img_lb.split('_')]
        new_lb = [ '0'+str(r) if r<10 else str(r) for r in lb]
        if len(new_lb)==2:
            items = int(new_lb[0]+new_lb[1])
        if len(new_lb)==3:
            items = int(new_lb[0]+new_lb[1]+new_lb[2])
        if len(new_lb)==4:
            items = int(new_lb[0]+new_lb[1]+new_lb[2]+new_lb[3])
        sort_lines[items] = line
    # print(sort_lines)
    sort_lines = dict(sorted(sort_lines.items()))
    new_lines = list(sort_lines.values())
    return new_lines

##Showing All Transitions...

Defining a function to draw images by iteration of a given directory. 

In [ ]:
def show_transitions(f_name):
  filename = f_name
  crop_line = os.listdir(filename)
  crop_lines = line_sort(crop_line)
  print(crop_lines)
  j = 1
  for i in crop_lines:
    print("Line no:",j)
    print("Image name:: ",i)
    im = filename + "/" + i
    crop_img = cv2.imread(im)
    
    width = crop_img.shape[1]
    height = crop_img.shape[0]
    print("Width, height: ",width, height)

    plot_fig(crop_img)
    j = j + 1

Defining a function to plot figures.

In [ ]:
def plot_fig(img, size = 15):
  plt.figure(figsize=(size, size))
  plt.imshow(imutils.opencv2matplotlib(img))
  plt.show()

Defining a function to visually compare results of annotations and predictions of each line image by plotting them one after another.

In [ ]:
def show_transitions_by_comparing(filename, gt_source_path, name1, name2):
    crop_line = os.listdir(filename)
    crop_lines = line_sort(crop_line)
    print("Cropped Line images: ",crop_lines)
    print()
    j = 1
    for i in crop_lines:
      print("Line no:",j)
      print("Image name:: ",i)
      im = filename + "/" + i
      crop_img = cv2.imread(im)
      
      width = crop_img.shape[1]
      height = crop_img.shape[0]
      
      # print("Annotated Line or GroundTruth ->")
      print(name1)
      # img_gd = gt_line_img_dir + i
      img_gd = gt_source_path + i
      img_gd1 = cv2.imread(img_gd)
      plot_fig(img_gd1)

      # print("Predicted Line ->")
      print(name2)
      plot_fig(crop_img)
      print()
      j = j + 1

##Drawing bounding box...

Defining a function to draw bounding boxes on images with their given labels.

In [ ]:
def draw_BB(img_path, label_path, flag):
  img = cv2.imread(img_path)
  dh, dw, _ = img.shape

  lb = open(label_path, 'r')
  data = lb.readlines()
  lb.close()

  for dt in data:
    if flag == 0:
      _, x, y, w, h = map(float, dt.split(' '))
    else:
      _, x, y, w, h, conf = map(float, dt.split(' '))
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    if flag == 0:
      cv2.rectangle(img, (l, t), (r, b), (0, 250, 0), 2)
    else:
      cv2.rectangle(img, (l, t), (r, b), (0, 0, 250), 2)
    
  # plot_fig(img)
  return img


#**Detecting lines with YOLO** (Upload the given trained Line Model weights on your drive and assign its directory to the --weights command)

Defining a function where we pass necessary attributes to detect lines by YOLO.

In [ ]:
# Yolo Detection...
def yolo_detection(img_path, img_size, conf):
  # %cd yolov5
  !python /content/yolov5/detect.py --weights /content/drive/MyDrive/thesis/yolov5/temp/YOLO_5x6_model/Line/best.pt --img $img_size --conf $conf --source $img_path --save-conf --save-txt

##Sorting & Filtering lines & words ( labels ) after their 1st detection...

Defining a function to sort and filter -

* lines based on their y-axis's attribute and confidence, respectively. 
* words based on their x-axis's attribute and width, respectively.  




In [ ]:
class Line_sort:
    def __init__(self, txt_files, txt_loc, sort_label, flag):
        self.txt_files = txt_files
        self.txt_loc = txt_loc
        self.sort_label = sort_label
        self.flag = flag
        self.read_file()

    def read_file(self):
        files = self.txt_files
        # os.mkdir('/content/sorted_line_after_1st_detection')
        os.mkdir(self.sort_label)
        for file in files:
            txt_file = []
            file_loc = self.txt_loc+file

            with open(file_loc, 'r' , encoding='utf-8',errors='ignore') as lines:
                for line in lines:
                    token = line.split()
                    
                    _, x, y, w, h, conf = map(float, line.split(' '))
                    # print("width -> ",w)
                    # print("confidence -> ",conf)
                    if self.flag == 0: # 1st line detection lavel
                      if w > 0.50 and conf < 0.50:
                        continue
                      else:
                        txt_file.append(token)
                    else: # Word detection lavel
                      # if w > 0.50:
                      #   continue
                      # else:
                        txt_file.append(token)

            if self.flag == 0: # 1st line detection lavel
               sorted_txt_file = sorted(txt_file, key=operator.itemgetter(2))
            else: # Word detection lavel
               sorted_txt_file = sorted(txt_file, key=operator.itemgetter(1))

            # lenght = len(sorted_txt_file[0])
            self.file_write(sorted_txt_file, file)

    def file_write(self,txt_file, file_name):
        # loc = '/content/sorted_line_after_1st_detection/'+file_name
        loc = self.sort_label+file_name
        with open(loc, 'w') as f: 
            c=0
            for line in txt_file:  
                for l in line:
                    c+=1
                    if c == len(line):
                        f.write('%s' % l)
                    else:
                        f.write('%s ' % l)
                f.write("\n")
                c=0


In [ ]:
def sort_detection_label(txt_loc, sort_label, flag):
  txt_files = os.listdir(txt_loc)
  obj = Line_sort(txt_files, txt_loc, sort_label, flag)
  

##1st Line Segment from Sorted line ( labels ) after 1st detection...

Defining a function to segment lines with YOLO's 1st line detection where we take full document image's width for a line if detected lines width ranges between 50% to 80%; otherwise, the lines are segmented as per detection.

In [ ]:
# text file and image sorting
def images_and_txtfile_sort(images, txt_file):
    txt = []
    image = []
    for item in txt_file:
        ch = ""
        for c in item:
            if c == "_":
                break
            ch += c
        txt.append(ch)

    txt.sort(key=int)
    sorted_txtfiles = []
    sorted_images = []

    for i in range(len(txt)):
        for ele, ele2 in zip(txt_file, images):
            st = ""
            st2 = ""
            for ch in ele:
                if ch == "_":
                    break
                st += ch
            if st == txt[i]:
                sorted_txtfiles.append(ele)
            for ch in ele2:
                if ch == "_":
                    break
                st2 += ch
            if st2 == txt[i]:
                sorted_images.append(ele2)
    return sorted_images, sorted_txtfiles

# line segmantation by yolo bounding box
def line_segmantation(path, image_loc, txt_loc, images, txt_file):

    # image, txt_files = images_and_txtfile_sort(images, txt_file)
    image = line_sort(images)
    txt_files = line_sort(txt_file)
    print(image)
    print(txt_files)
    
    for image, txt in zip(image, txt_files):
        
        # make folder name according to txt file
        new_folder = txt[:-4]
        new_folder_loc = path + new_folder
        # create subfolder in Lines according to txt file
        os.mkdir(new_folder_loc)
        # current image location
        current_image = image_loc + image
        img = cv2.imread(current_image)
        dh, dw, _ = img.shape
        # current txt file location
        current_txt = txt_loc + txt
        fl = open(current_txt, 'r')
        data = fl.readlines()
        fl.close()
        k=1
        for dt in data:
            # _, x, y, w, h = map(float, dt.split(' '))
            _, x, y, w, h,  conf = map(float, dt.split(' '))
            if w > 0.50 and w < 0.80:
               x = 0.5
               w = 1.0
            l = int((x - w / 2) * dw)
            r = int((x + w / 2) * dw)
            t = int((y - h / 2) * dh)
            b = int((y + h / 2) * dh)

            crop = img[t:b, l:r]
            cv2.imwrite("{}{}/{}_{}.jpg".format(path, new_folder, new_folder, k), crop)
            k += 1

def take_valid_img(images):
    image = []
    valid_img_ext = ["jpg", "JPG", "jpeg", "JPEG", "png", "PNG"]
    for img in images:
        try:
            ext = img.split('.')[1]
            if ext not in valid_img_ext :
                continue
            else:
                image.append(img)
        except:
            continue
    return image


In [ ]:
def line_segmentation(img_path, sorted_label):
  current_directory = img_path
  #print(current_directory)
  current_directory1 = sorted_label
  #print(current_directory1)
  images = []  # take for all images
  txt = []     # take for all txt files

  # images
  # take all files in list in current directory location
  current_directory_files = os.listdir(current_directory) 
  #print(current_directory_files)
  if not current_directory_files:
      print("{} folder is empty!")

  # lines
  # take all files in list in current directory1 location
  current_directory_1_files = os.listdir(current_directory1)
  if not current_directory_1_files:
      print("Line folder is empty of {} !")

  txt_pattern = "*.txt"
  # take all only image in images list from current directory files list
  images1 = [entry for entry in current_directory_files]
  images = take_valid_img(images1)

  # take all only txt file in txt list from current directory1 files list
  txt = [entry for entry in current_directory_1_files if fnmatch.fnmatch(entry, txt_pattern)]

  os.mkdir('/content/croped_line_after_1st_detection')
  path = "/content/croped_line_after_1st_detection/"
  # calling the line segmantation function
  line_segmantation(path, current_directory, current_directory1, images, txt)
  print("Successful line segmantated in the {} folder".format(path))

#**Rotation**

##DSkew by HaughLine...

Defining functions to do necessary preprocessing and calculating a line image's dimension skew (DSkew) using Probabilistic Hough Line Transform (PHT). And correcting it by using the Affine Transform if the DSkew angle detected by PHT is more than 0; otherwise, keep the image without DSkew by setting the DSkew angle as 0.  

In [ ]:
class ImgCorrect():
    def __init__(self, img):
        self.img = img
        self.h, self.w, self.channel = self.img.shape
        # print("Original images h & w -> | w: ",self.w, "| h: ",self.h)
        if self.w <= self.h:
            self.scale = 700 / self.w
            self.img = cv2.resize(self.img, (0, 0), fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        else:
            self.scale = 700 / self.h
            self.img = cv2.resize(self.img, (0, 0), fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        # print("Scaled image:")
        # plot_fig(self.img)
        self.gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)

    def img_lines(self):
        # print("Gray Image:")
        # plot_fig(self.gray)
        ret, binary = cv2.threshold(self.gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        # print("Inverse Binary:")
        # plot_fig(binary)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # rectangular structure
        # print("Kernel for dialation:")
        # print(kernel)
        binary = cv2.dilate(binary, kernel)  # dilate
        # print("Dialated Binary:")
        # plot_fig(binary)
        edges = cv2.Canny(binary, 50, 200)
        # print("Canny edged detection:")
        # plot_fig(edges)

        self.lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=20)
        # print(self.lines)
        if self.lines is None:
            print("Line segment not found")
            return None

        lines1 = self.lines[:, 0, :]  # Extract as 2D
        imglines = self.img.copy()
        for x1, y1, x2, y2 in lines1[:]:
            cv2.line(imglines, (x1, y1), (x2, y2), (0, 255, 0), 3)
        # print("Probabilistic Hough Lines:")
        # plot_fig(imglines)
        # return imglines

    def search_lines(self):
      lines = self.lines[:, 0, :]  # extract as 2D
    
      number_inexist_k = 0
      sum_pos_k45 = number_pos_k45 = 0
      sum_pos_k90 = number_pos_k90 = 0
      sum_neg_k45 = number_neg_k45 = 0
      sum_neg_k90 = number_neg_k90 = 0
      sum_zero_k = number_zero_k = 0

      for x in lines:
          if x[2] == x[0]:
              number_inexist_k += 1
              continue
          #print(degrees(atan((x[3] - x[1]) / (x[2] - x[0]))), "pos:", x[0], x[1], x[2], x[3], "Slope:",(x[3] - x[1]) / (x[2] - x[0]))
          degree = degrees(atan((x[3] - x[1]) / (x[2] - x[0])))
          # print("Degree or Slope of detected lines : ",degree)
          if 0 < degree < 45:
              number_pos_k45 += 1
              sum_pos_k45 += degree
          if 45 <= degree < 90:
              number_pos_k90 += 1
              sum_pos_k90 += degree
          if -45 < degree < 0:
              number_neg_k45 += 1
              sum_neg_k45 += degree
          if -90 < degree <= -45:
              number_neg_k90 += 1
              sum_neg_k90 += degree
          if x[3] == x[1]:
              number_zero_k += 1

      max_number = max(number_inexist_k, number_pos_k45, number_pos_k90, number_neg_k45,number_neg_k90, number_zero_k)
      # print("Num of lines in different Degree range ->")
      # print("Not a Line: ",number_inexist_k, "| 0 to 45: ",number_pos_k45, "| 45 to 90: ",number_pos_k90, "| -45 to 0: ",number_neg_k45, "| -90 to -45: ",number_neg_k90, "| Line where y1 equals y2 :",number_zero_k)
    
      if max_number == number_inexist_k:
          return 90
      if max_number == number_pos_k45:
          return sum_pos_k45 / number_pos_k45
      if max_number == number_pos_k90:
          return sum_pos_k90 / number_pos_k90
      if max_number == number_neg_k45:
          return sum_neg_k45 / number_neg_k45
      if max_number == number_neg_k90:
          return sum_neg_k90 / number_neg_k90
      if max_number == number_zero_k:
          return 0

    def rotate_image(self, degree):
        """
        Positive angle counterclockwise rotation
        :param degree:
        :return:
        """
        # print("degree:", degree)
        if -45 <= degree <= 0:
            degree = degree  # #negative angle clockwise
        if -90 <= degree < -45:
            degree = 90 + degree  # positive angle counterclockwise
        if 0 < degree <= 45:
            degree = degree  # positive angle counterclockwise
        if 45 < degree < 90:
            degree = degree - 90  # negative angle clockwise
        print("DSkew angle: ", degree)

        # degree = degree - 90
        height, width = self.img.shape[:2]
        heightNew = int(width * fabs(sin(radians(degree))) + height * fabs(
            cos(radians(degree))))  # This formula refers to the previous content
        widthNew = int(height * fabs(sin(radians(degree))) + width * fabs(cos(radians(degree))))
        # print("Height :",height)
        # print("Width :",width)
        # print("HeightNew :",heightNew)
        # print("WidthNew :",widthNew)

        matRotation = cv2.getRotationMatrix2D((width / 2, height / 2), degree, 1)  # rotate degree counterclockwise
        # print("Mat Rotation (Before): ",matRotation)
        matRotation[0, 2] += (widthNew - width) / 2
        # Because after rotation, the origin of the coordinate system is the upper left corner of the new image, so it needs to be converted according to the original image
        matRotation[1, 2] += (heightNew - height) / 2
        # print("Mat Rotation (After): ",matRotation)

        # Affine transformation, the background color is filled with white
        imgRotation = cv2.warpAffine(self.img, matRotation, (widthNew, heightNew), borderValue=(255, 255, 255))

        # Padding
        pad_image_rotate = cv2.warpAffine(self.img, matRotation, (widthNew, heightNew), borderValue=(0, 255, 0))
        # plot_fig(pad_image_rotate)

        return imgRotation

def dskew(line_path, img):
    img_loc = line_path + img
    im = cv2.imread(img_loc)

    # Padding
    bg_color = [255, 255, 255]
    pad_img = cv2.copyMakeBorder(im,100,100,100,100,cv2.BORDER_CONSTANT,value=bg_color)

    imgcorrect = ImgCorrect(pad_img)
    lines_img = imgcorrect.img_lines()
    # print(type(lines_img))
    
    if lines_img is None:
        rotate = imgcorrect.rotate_image(0)
    else:
        degree = imgcorrect.search_lines()
        rotate = imgcorrect.rotate_image(degree)


    return rotate


## HoughLine and Affine Transform...

Defining functions to do necessary preprocessing and calculating the skew of an image's main handwritten line using Standard Hough Line Transform (SHT). And correcting it by using Affine Transform if the skew angle detected by SHT is more than 0; otherwise, send the image for DSkew. 

In [ ]:
rotate_line = "/content/Rotated_line_by_HaughLine_Affine/"
os.mkdir(rotate_line)

rotate_line_Dskew = "/content/DSkew/"
os.mkdir(rotate_line_Dskew)

rotate_line_Haughline = "/content/HaughLine_Affine/"
os.mkdir(rotate_line_Haughline)

exception = []

# Degree conversion
def DegreeTrans(theta):
    res = theta / np.pi * 180
    # print(res)
    return res

# Rotate the image degree counterclockwise (original size)
def rotateImage(src, degree):
    # The center of rotation is the center of the image
    h, w = src.shape[:2]
    # Calculate the two-dimensional rotating affine transformation matrix
    RotateMatrix = cv2.getRotationMatrix2D((w / 2.0, h / 2.0), degree, 1)
    # print("Rotate Matrix: ")
    # print(RotateMatrix)

    # Affine transformation, the background color is filled with GREEN so that the rotation can be easily understood
    rotate1 = cv2.warpAffine(src, RotateMatrix, (w, h), borderValue=(0, 255, 0))
    # plot_fig(rotate1)
    # Affine transformation, the background color is filled with white
    rotate = cv2.warpAffine(src, RotateMatrix, (w, h), borderValue=(255, 255, 255))

    # Padding
    bg_color = [255, 255, 255]
    pad_image_rotate = cv2.copyMakeBorder(rotate,100,100,100,100,cv2.BORDER_CONSTANT,value=bg_color)

    return pad_image_rotate

# Calculate angle by Hough transform
def CalcDegree(srcImage,canny_img):
    lineimage = srcImage.copy()
    lineimg = srcImage.copy()
    # Detect straight lines by Hough transform
    # The fourth parameter is the threshold, the greater the threshold, the higher the detection accuracy
    try:
        lines = cv2.HoughLines(canny_img, 1, np.pi / 180, 200)
        # print("HoughLines: ")
        # cv2_imshow(lines)
        # Due to different images, the threshold is not easy to set, because the threshold is set too high, so that the line cannot be detected, the threshold is too low, the line is too much, the speed is very slow
        theta_sum = 0
        rho_sum = 0
        sum_x1 = sum_x2 = sum_y1 = sum_y2 = 0
        # Draw each line segment in turn
        for i in range(len(lines)):
            for rho, theta in lines[i]:
                # print("theta:", theta, " rho:", rho)
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(round(x0 + 1000 * (-b)))
                y1 = int(round(y0 + 1000 * a))
                x2 = int(round(x0 - 1000 * (-b)))
                y2 = int(round(y0 - 1000 * a))
                # print("a: ",a, " b: ",b, " x0: ",x0, " y0: ",y0, " x1: ",x1, " y1: ",y1, " x2: ",x2, " y2: ",y2)
                # Only select the smallest angle as the rotation angle
                sum_x1+=x1
                sum_x2+=x2
                sum_y1+=y1
                sum_y2+=y2
                rho_sum += rho
                theta_sum += theta
                cv2.line(lineimage, (x1, y1), (x2, y2), (0, 0, 255), 1, cv2.LINE_AA)

        
        # print("HoughLines: ")
        # plot_fig(lineimage)
        print()

        pt1 = (sum_x1//len(lines), sum_y1//len(lines))
        pt2 = (sum_x2//len(lines), sum_y2//len(lines))
        
        # print("Sum of thetas: ",theta_sum)
        # print("lines: ",lines)
        average = theta_sum / len(lines)
        # print("Avg. Theta: ",average)
        angle = DegreeTrans(average) - 90
        # print("Avg. Angle: ",angle)
        print("Skewed Angle: ",angle)
        average_rho = rho_sum / len(lines)
        # print("Avg. rho: ",average_rho)

        # print('Draw best fit line with full:')
        # h, w = lineimg.shape[:2]
        # pt2 = (w,h)
        # print("Cordinates of the best fit line: ",pt1,pt2)
        cv2.line(lineimg, pt1, pt2, (0,0,255), 2)
        # plot_fig(lineimg)

        return angle
    except:
        angle = 0.0
        return angle

def ready_for_rotate(line_path, img):
    print()
    print("Image :: ",img)
    img_loc = line_path + img
    image = cv2.imread(img_loc)

    org_width = image.shape[1]
    org_height = image.shape[0]

    img1 = image
    im_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    # print("Gray Image: ")
    # plot_fig(im_gray)

    edges = cv2.Canny(im_gray,50,150,apertureSize=3)
    # print("Canny Image: ")
    # plot_fig(edges)

    degree = CalcDegree(image,edges)
    
    if degree == 0.0:
        rotate = dskew(line_path, img)
        # print("Rotated Image by DSkew: ")
        # plot_fig(rotate)
        print()
        
        filename1 = rotate_line_Dskew + img
        cv2.imwrite(filename1, rotate)
        filename = rotate_line + img
        cv2.imwrite(filename, rotate)
    else:
        rotate = rotateImage(image, degree)
        # print("Rotated Image by Houghline Affine transform: ")
        # plot_fig(rotate)
        print()

        filename2 = rotate_line_Haughline + img
        cv2.imwrite(filename2, rotate)
        filename = rotate_line + img
        cv2.imwrite(filename, rotate)


In [ ]:
def rotate_lines(first_detection):
  line_path = first_detection
  line_dir = line_sort(os.listdir(line_path))
  print(line_dir)

  for img in line_dir:
      ready_for_rotate(line_path, img)


#**Final Line Segmentation**

##Find undetected image in Yolo 2nd detection...

Defining a function to get those images that are not getting 2nd YOLO detection for their low dimension and putting those images with the final segmented images.  

In [ ]:
undetected_images_path = []

def find_undetected_images(img, label):
  # img_path = "/content/Rotated_line_by_HaughLine_Affine/"
  img_path = img
  # detect_lb_path = "/content/yolov5/runs/detect/exp2/labels/"
  detect_lb_path = label
  undetect_img_path = "/content/final_line_segmentation/"

  def take_valid_img(images):
      image = []
      valid_img_ext = ["jpg", "JPG", "jpeg", "JPEG", "png", "PNG"]
      for img in images:
          try:
              ext = img.split('.')[1]
              if ext not in valid_img_ext :
                  continue
              else:
                  image.append(img)
          except:
              continue
      return image

  img1 = os.listdir(img_path)
  img = take_valid_img(img1)
  detect_lb = os.listdir(detect_lb_path)

  def find_undetect_img(img,detect_lb):
      img_lb = [im.split('.')[0] for im in img]
      dt_lb = [dt.split('.')[0] for dt in detect_lb]
      undt_lb = list(set(img_lb).difference(dt_lb))
      undetect_img = []
      detect_img = []
      for lb in undt_lb:
          for im in img:
              im_lb = im.split('.')[0]
              if lb == im_lb:
                  undetect_img.append(im)
              else:
                  detect_img.append(im)
      print("Undetect image: ",undetect_img)
      write_image(undetect_img)

  def write_image(undt_img):
      for im in undt_img:
          filename = undetect_img_path+im
          img = cv2.imread(img_path+im)
          cv2.imwrite(filename,img)
          undetected_images_path.append(filename)

  find_undetect_img(img,detect_lb)


## 2nd Line Segmentation after Rotation...

Defining a function to select the main handwritten line from line images followed by segmenting it using YOLO's 2nd line detection. 

In [ ]:
def crop_image(bb_data, destination, image, img_lb, dh, dw):
    x = float(bb_data[1])
    y = float(bb_data[2])
    w = float(bb_data[3])
    h = float(bb_data[4])
  
    # x = 0.5
    # w  = 1.0
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)

    crop = image[t:b, l:r]
    filename = destination+img_lb
    cv2.imwrite(filename,crop)
    print("Segmented successfully!\n")

def line_segmantation_2(img, img_path, label, label_path, segmented_img_path):
  dir = segmented_img_path
  print("Image path -> ",img_path)
  img1 = cv2.imread(img_path)
  dh, dw, _ = img1.shape
  txt_lb = open(label_path, 'r')
  txt_lb_data = txt_lb.readlines()
  txt_lb.close()
  img_name = img
  
  max_w = 0
  data1 = []
  for line in txt_lb_data:
      token = line.split()
      data1.append(token)
  
  if len(data1)==1:
      bb_data = data1[0]
      wdth = float(bb_data[3])
      if wdth>0.4:
          crop_image(bb_data,dir,img1,img_name,dh,dw,)
      else:
          filename = dir+img_name
          cv2.imwrite(filename,img1)
  elif len(data1)==2:
      bb_data1 = data1[0]
      bb_data2 = data1[1]
      w1 = float(bb_data1[3]) 
      w2 = float(bb_data2[3])
      c1 = float(bb_data1[5]) 
      c2 = float(bb_data2[5])
      if w1 <= 0.5 and w2 <= 0.5:
        if c1 >= 0.8 and c2 >= 0.8:
          sorted_bb_data = sorted(data1, key=operator.itemgetter(5))
          bb_data = sorted_bb_data[-1]
          crop_image(bb_data,dir,img1,img_name,dh,dw,)
        else:
          filename = dir+img_name
          cv2.imwrite(filename,img1)
      else:
        sorted_bb_data = sorted(data1, key=operator.itemgetter(3))
        bb_data = sorted_bb_data[-1]
        crop_image(bb_data,dir,img1,img_name,dh,dw,)
  elif len(data1)==3:
      sorted_bb_data = sorted(data1, key=operator.itemgetter(2))
      bb_data = sorted_bb_data[1]
      crop_image(bb_data,dir,img1,img_name,dh,dw,) 
  else:
      sorted_bb_data = sorted(data1, key=operator.itemgetter(3))
      bb_data = sorted_bb_data[-1]
      crop_image(bb_data,dir,img1,img_name,dh,dw,)



#**Word Segmentation**

Defining a function to segment the words of line images by using YOLO's word detection.

In [ ]:
def word_segmentation(line_images, word_labels):
  line_img = os.listdir(line_images)
  word_label = os.listdir(word_labels)
  print(line_img)
  print(word_label)

  for i in word_label:
    if i == "163_19_11.txt":
      print("Yes")
    for j in line_img:
      if j == "163_19_11.jpg":
         print("Yes")
      fn_i = i.split(".")
      fn_j = j.split(".")
      if fn_i[0] ==  fn_j[0]:
        # print("yes")
        dir = "/content/final_word_segmentation/"+fn_i[0]
        os.mkdir(dir)

        img = cv2.imread(line_images + j)
        dh, dw, _ = img.shape
        txt_lb = open(word_labels + i, 'r')
        txt_lb_data = txt_lb.readlines()
        txt_lb.close()
        img_lb = fn_i[0]
        
        k=1
        for dt in txt_lb_data:
            # _, x, y, w, h = map(float, dt.split(' '))
            _, x, y, w, h, conf = map(float, dt.split(' '))
            l = int((x - w / 2) * dw)
            r = int((x + w / 2) * dw)
            t = int((y - h / 2) * dh)
            b = int((y + h / 2) * dh)

            crop = img[t:b, l:r]
            cv2.imwrite("{}/{}_{}.jpg".format(dir, img_lb, k), crop)
            k += 1


#**Automatic annotation Starts from here** (Change File Path)

Uplaoded your dataset of raw images by following given "Dataset2.zip" format on your drive location. Then, assign the zip dataset's name and its directory to define variable **files1** and **files2**, respectively.  

In [ ]:
files1 = 'Dataset2.zip'
files2 = '/content/drive/MyDrive/Thesis_work/unannotated_dataset/Dataset2.zip'
print("File path: ",files2)
file_unzip(files2)

File path:  /content/drive/MyDrive/Thesis_work/unannotated_dataset/Dataset2.zip
Archive:  /content/drive/MyDrive/Thesis_work/unannotated_dataset/Dataset2.zip
   creating: /content/Dataset2/172/
  inflating: /content/Dataset2/172/172.pdf  
  inflating: /content/Dataset2/172/172.txt  
  inflating: /content/Dataset2/172/172_1.jpg  
  inflating: /content/Dataset2/172/172_2.jpg  
  inflating: /content/Dataset2/172/172_3.jpg  
  inflating: /content/Dataset2/172/172_4.jpg  
  inflating: /content/Dataset2/172/172_5.jpg  
  inflating: /content/Dataset2/172/172_6.jpg  
  inflating: /content/Dataset2/172/172_7.jpg  
  inflating: /content/Dataset2/172/172_8.jpg  
   creating: /content/Dataset2/173/
  inflating: /content/Dataset2/173/173.txt  
  inflating: /content/Dataset2/173/173_1.jpg  
  inflating: /content/Dataset2/173/173_2.jpg  
  inflating: /content/Dataset2/173/173_3.jpg  
  inflating: /content/Dataset2/173/173_4.jpg  
  inflating: /content/Dataset2/173/173_5.jpg  
   creating: /content/Da

This section is performing these things provide below:
1. Renaming the extracted file as 'files'.
2. Going into each folder of the dataset, taking the document image labels and path followed by putting them into dictionary for later use.
3. Also, creating folders to store automation annotated lines and words by our system.

In [ ]:
def filter_list(list):
  list1 = []
  for i in list:
    if i.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif', 'JPG', 'JPEG', 'PNG')) == True:
      list1.append(i)
  return list1

dictonary1 = {}
dictonary2 = {}

# renaming extracted file as 'files'
fname = files1.split('.')[0]
src_dir = '/content/' + fname
# src_dir = '/content/Dataset2'
dst_dir = '/content/files'
os.rename(src_dir, dst_dir)
final_dir = dst_dir
print(final_dir)

# final_dir = '/content/' + files1.split('.')[0]
# print(final_dir)

file_list = os.listdir(final_dir)
for i in file_list:
  final_dir1 = final_dir + '/' + i + '/'
  print(final_dir1)
  final_dir1_list = filter_list(os.listdir(final_dir1))
  final_dir1_list1 = line_sort(final_dir1_list)
  print(final_dir1_list1)
  dictonary1[final_dir1] = final_dir1_list1
  dictonary2[i] = final_dir1_list1

  final_dir2_line = final_dir1 + 'lines/'
  final_dir2_word = final_dir1 + 'words/'
  os.mkdir(final_dir2_line)
  os.mkdir(final_dir2_line + 'images')
  os.mkdir(final_dir2_line + 'labels')
  os.mkdir(final_dir2_word)
  os.mkdir(final_dir2_word + 'images')
  os.mkdir(final_dir2_word + 'labels')
print(dictonary1)

/content/files
/content/files/173/
['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg']
/content/files/174/
['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg']
/content/files/172/
['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']
{'/content/files/173/': ['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg'], '/content/files/174/': ['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg'], '/content/files/172/': ['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']}


In [ ]:
def copy_files(path, files):
  for i in files:
    path1 = path + i
    print(path1)
    img = cv2.imread(path1)
    # cv2_imshow(img)
    cv2.imwrite(os.path.join(dir, i), img)
dir = '/content/temporary_files/'
os.mkdir(dir)
for key in dictonary1.keys():
  val = dictonary1.get(key)
  print(key,'->',val)
  copy_files(key, val)

/content/files/173/ -> ['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg']
/content/files/173/173_1.jpg
/content/files/173/173_2.jpg
/content/files/173/173_3.jpg
/content/files/173/173_4.jpg
/content/files/173/173_5.jpg
/content/files/174/ -> ['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg']
/content/files/174/174_1.jpg
/content/files/174/174_2.jpg
/content/files/174/174_3.jpg
/content/files/174/174_4.jpg
/content/files/172/ -> ['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']
/content/files/172/172_1.jpg
/content/files/172/172_2.jpg
/content/files/172/172_3.jpg
/content/files/172/172_4.jpg
/content/files/172/172_5.jpg
/content/files/172/172_6.jpg
/content/files/172/172_7.jpg
/content/files/172/172_8.jpg


First line detection of document image by YOLO. 

In [ ]:
# 1st detection...
img_path = dir
img_size = 640
conf = 0.30
yolo_detection(img_path, img_size, conf)

detect: weights=['/content/drive/MyDrive/thesis/yolov5/temp/YOLO_5x6_model/Line/best.pt'], source=/content/temporary_files/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-67-g2b356c0 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86173414 parameters, 0 gradients
image 1/17 /content/temporary_files/172_1.jpg: 640x480 21 lines, 73.7ms
image 2/17 /content/temporary_files/172_2.jpg: 640x480 20 lines, 60.6ms
image 3/17 /content/temporary_files/172_3.jpg: 640x480 26 lines, 60.6ms
image 4/17 /content/temporary_files/172_4.jpg: 640x480 26 lines, 60.6ms
image 5/17 /c

In [ ]:
print(dictonary2)
# Sorting Labels of 1st detection on the basis of y...
txt_loc = "/content/yolov5/runs/detect/exp/labels/"
new_sort_label = '/content/sorted_line_after_1st_detection/'
flag = 0
sort_detection_label(txt_loc, new_sort_label, flag)

{'173': ['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg'], '174': ['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg'], '172': ['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']}


In [ ]:
# Line Segmentation after 1st Detection...
img_path = '/content/temporary_files/'
sorted_label = "/content/sorted_line_after_1st_detection/"
line_segmentation(img_path, sorted_label)

['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg', '173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg', '174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg']
['172_1.txt', '172_2.txt', '172_3.txt', '172_4.txt', '172_5.txt', '172_6.txt', '172_7.txt', '172_8.txt', '173_1.txt', '173_2.txt', '173_3.txt', '173_4.txt', '173_5.txt', '174_1.txt', '174_2.txt', '174_3.txt', '174_4.txt']
Successful line segmantated in the /content/croped_line_after_1st_detection/ folder


In [ ]:
dictonary3 = {}
for key in dictonary2.keys():
  path1 = '/content/files/' + key + '/lines/images/'
  path2 = '/content/files/' + key + '/words/images/'
  lb_path = '/content/files/' + key + '/lines/labels/'

  temp_list = []
  val = dictonary2.get(key)
  for i in val:
    temp1 = i.split('.')[0]
    temp_list.append(temp1)

    line_path = path1 + temp1 + '/'
    os.mkdir(line_path)

    word_path = path2 + temp1 + '/'
    os.mkdir(word_path)

    # from_dir = "/content/croped_line_after_1st_detection/" + temp1
    # to_dir = path1
    # shutil.copytree(from_dir, to_dir)

    from_dir_lb = '/content/sorted_line_after_1st_detection/' + temp1 +'.txt'
    to_dir_lb = lb_path + temp1 +'.txt'
    shutil.copy(from_dir_lb, to_dir_lb)

  dictonary3[key] = temp_list


In [ ]:
path1 = '/content/croped_line_after_1st_detection/'
path2 = '/content/croped_line_after_1st_detection_1/'
os.mkdir(path2)
temp1 = os.listdir(path1)
for i in temp1:
  temp2 = path1 + i + '/'
  temp3 = os.listdir(temp2)
  for j in temp3:
    temp4 = temp2 + j
    img = cv2.imread(temp4)
    filename = path2 + j
    cv2.imwrite(filename, img)
# for key in dictonary3.keys():
#   val = dictonary3.get(key)
#   print(key,"->",val)

Rotating segmented lines.

In [ ]:
first_detection = '/content/croped_line_after_1st_detection_1/'
rotate_lines(first_detection)

['172_1_1.jpg', '172_1_2.jpg', '172_1_3.jpg', '172_1_4.jpg', '172_1_5.jpg', '172_1_6.jpg', '172_1_7.jpg', '172_1_8.jpg', '172_1_9.jpg', '172_1_10.jpg', '172_1_11.jpg', '172_1_12.jpg', '172_1_13.jpg', '172_1_14.jpg', '172_1_15.jpg', '172_1_16.jpg', '172_1_17.jpg', '172_1_18.jpg', '172_1_19.jpg', '172_1_20.jpg', '172_2_1.jpg', '172_2_2.jpg', '172_2_3.jpg', '172_2_4.jpg', '172_2_5.jpg', '172_2_6.jpg', '172_2_7.jpg', '172_2_8.jpg', '172_2_9.jpg', '172_2_10.jpg', '172_2_11.jpg', '172_2_12.jpg', '172_2_13.jpg', '172_2_14.jpg', '172_2_15.jpg', '172_2_16.jpg', '172_2_17.jpg', '172_2_18.jpg', '172_2_19.jpg', '172_2_20.jpg', '172_3_1.jpg', '172_3_2.jpg', '172_3_3.jpg', '172_3_4.jpg', '172_3_5.jpg', '172_3_6.jpg', '172_3_7.jpg', '172_3_8.jpg', '172_3_9.jpg', '172_3_10.jpg', '172_3_11.jpg', '172_3_12.jpg', '172_3_13.jpg', '172_3_14.jpg', '172_3_15.jpg', '172_3_16.jpg', '172_3_17.jpg', '172_3_18.jpg', '172_3_19.jpg', '172_3_20.jpg', '172_4_1.jpg', '172_4_2.jpg', '172_4_3.jpg', '172_4_4.jpg', '172_4

Second line detection on segmented lines by YOLO.

In [ ]:
# Image Path from unziped file...
# Yolo 2nd Detection...
rotated_img_path = "/content/Rotated_line_by_HaughLine_Affine/"
img_size = 640
conf = 0.50
yolo_detection(rotated_img_path, img_size, conf)

detect: weights=['/content/drive/MyDrive/thesis/yolov5/temp/YOLO_5x6_model/Line/best.pt'], source=/content/Rotated_line_by_HaughLine_Affine/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-67-g2b356c0 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86173414 parameters, 0 gradients
image 1/296 /content/Rotated_line_by_HaughLine_Affine/172_1_1.jpg: 96x640 1 line, 27.2ms
image 2/296 /content/Rotated_line_by_HaughLine_Affine/172_1_10.jpg: 96x640 1 line, 21.0ms
image 3/296 /content/Rotated_line_by_HaughLine_Affine/172_1_11.jpg: 96x640 1 line, 21.1ms
image 4/296 /co

In [ ]:
new_dir = "/content/final_line_segmentation/"
os.mkdir(new_dir)

Extracting the main handwritten line from second YOLO line detection and those lines are the final segmented lines of our document images.

In [ ]:
target_label_path = "/content/yolov5/runs/detect/exp2/labels/"
target_image_path = "/content/Rotated_line_by_HaughLine_Affine/"
target_image = os.listdir(target_image_path)
target_label = os.listdir(target_label_path)

for i in target_image:
  for j in target_label:
    fn_i = i.split(".")
    fn_j = j.split(".")
    if fn_i[0] ==  fn_j[0]:
      # Line Segmentation after 1st Detection...
      # Final Line Segmentation...
      img_path = target_image_path + i
      img = i
      sorted_label = j
      sorted_label_path = target_label_path + j
      line_segmantation_2(img, img_path, sorted_label, sorted_label_path, new_dir)

Image path ->  /content/Rotated_line_by_HaughLine_Affine/172_2_11.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/173_2_21.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/172_2_10.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/173_2_10.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/174_1_1.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/173_3_15.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/174_2_9.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/172_4_12.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/172_5_3.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Affine/172_1_18.jpg
Segmented successfully!

Image path ->  /content/Rotated_line_by_HaughLine_Aff

Undetected lines in YOLO second line detection.

In [ ]:
print("List of undetected images in 2nd detection ->")
find_undetected_images(target_image_path, target_label_path)
print()
# print("Undetected images are shown below -> \n")
# for i in list(set(undetected_images_path)):
#   print("Line image path: ",i)
#   undit_img = cv2.imread(i)
#   plot_fig(undit_img)
#   print()

List of undetected images in 2nd detection ->
Undetect image:  ['173_3_7.jpg', '173_2_17.jpg', '173_5_5.jpg', '174_3_11.jpg', '174_3_7.jpg', '173_2_16.jpg', '173_3_18.jpg', '173_4_14.jpg', '174_2_10.jpg', '173_2_22.jpg', '173_2_13.jpg', '173_3_4.jpg', '174_2_6.jpg', '173_3_12.jpg', '173_3_8.jpg', '174_3_2.jpg', '173_3_5.jpg', '173_3_1.jpg', '173_1_9.jpg', '173_3_14.jpg', '173_3_13.jpg', '173_3_9.jpg', '173_1_19.jpg', '173_1_16.jpg', '173_3_2.jpg', '173_4_1.jpg', '174_4_6.jpg']



Trimming the DSkewed images from all the side to reduce the padding caused by scaling. 

In [ ]:
def trim_original_image(rotate, org_w, org_h):
  org_width = org_w
  org_height = org_h

  img1 = rotate
  width = img1.shape[1]
  height = img1.shape[0]
  print("Original height -> ",org_height)
  print("Original width -> ",org_width)

  start_row = 60
  end_row = height - 60

  start_col = 60
  end_col = width - 60
  img_new = img1[start_row:end_row, start_col:end_col]

  width1 = img_new.shape[1]
  height1 = img_new.shape[0]
  print("New height -> ",height1)
  print("New width -> ",width1)
    
  return img_new


final_line_segment = "/content/final_line_segmentation/"
rotate_line_Dskew = "/content/DSkew/"
dskew_img_list = os.listdir(rotate_line_Dskew)
print(dskew_img_list)
for i in dskew_img_list:
  print("Target image -> ",i)
  temp = final_line_segment + i
  print("Path -> ",temp)
  img1 = cv2.imread(temp)
  height, width, channels = img1.shape
  temp2 = rotate_line_Dskew + i
  img2 = cv2.imread(temp2)
  height2, width2, channels = img2.shape
  if height >= height2:
    # os.remove(temp)
    temp3 = trim_original_image(img1, width, height)
    cv2.imwrite(temp, temp3)
    # print("Original ->")
    # plot_fig(img1)
    # print("New ->")
    # plot_fig(temp3)
    print()
  # else:
  #   print("Original ->")
  #   plot_fig(img1)
  #   print("No need for change as its gone through the 2nd line detection!\n")


['173_2_21.jpg', '173_2_13.jpg', '173_2_10.jpg', '173_3_15.jpg', '173_4_7.jpg', '173_1_6.jpg', '173_4_5.jpg', '173_1_15.jpg', '173_4_16.jpg', '173_4_22.jpg', '174_4_6.jpg', '173_2_5.jpg', '174_3_11.jpg', '173_2_15.jpg', '173_3_18.jpg', '173_1_17.jpg', '173_1_10.jpg', '173_2_16.jpg', '173_2_19.jpg', '173_1_8.jpg', '173_2_9.jpg', '173_1_19.jpg', '173_3_14.jpg', '173_3_5.jpg', '173_3_13.jpg', '173_5_3.jpg', '173_2_1.jpg', '173_1_13.jpg', '173_4_20.jpg', '173_3_8.jpg', '173_4_14.jpg', '173_2_6.jpg', '173_1_5.jpg', '173_3_1.jpg', '173_4_8.jpg', '173_3_2.jpg', '173_1_21.jpg', '173_3_11.jpg', '173_3_3.jpg', '173_4_24.jpg', '173_2_14.jpg', '173_1_3.jpg', '173_1_16.jpg', '173_1_12.jpg', '173_4_23.jpg', '173_4_4.jpg', '173_1_22.jpg', '173_2_22.jpg', '173_1_2.jpg', '173_3_4.jpg', '174_1_6.jpg', '173_3_16.jpg', '173_1_18.jpg', '173_3_10.jpg', '174_4_9.jpg', '173_5_6.jpg', '173_2_8.jpg', '173_2_20.jpg', '173_1_4.jpg', '173_1_1.jpg', '173_1_14.jpg', '173_2_4.jpg', '173_4_2.jpg', '173_3_17.jpg', '173

In [ ]:
# Copying final segmented lines to the final folders (files)...
def trim_label(x):
  t1 = x.split('_')[0]
  t2 = x.split('_')[1]
  t3 = t1+"_"+t2
  return t3

target_dir = "/content/files/"
final_dir = "/content/final_line_segmentation/"
final_dir_list = os.listdir(final_dir)
for key in dictonary3.keys():
  values = dictonary3.get(key)
  for val in values:
    target_dir1 = target_dir + key + "/lines/images/" + val +"/"
    for i in final_dir_list:
      temp = trim_label(i)
      if val == temp:
        src_path = final_dir + i
        dst_path = target_dir1 + i
        shutil.copy(src_path, dst_path)

In [ ]:
# x = '/content/final_line_segmentation/'
# y = os.listdir(x)
# y1 = line_sort(y)
# for i in y1:
#   print(i)
#   z = x + i
#   im = cv2.imread(z)
#   cv2_imshow(im)
#   print()

# **Word detection on final segmented lines** (Upload the given trained Word Model weights on your drive and assign its directory to the --weights command.).

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/thesis/yolov5/temp/YOLO_5x6_model/Word/best.pt --img 640 --conf 0.40 --source /content/final_line_segmentation --save-conf --save-txt

detect: weights=['/content/drive/MyDrive/thesis/yolov5/temp/YOLO_5x6_model/Word/best.pt'], source=/content/final_line_segmentation, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-67-g2b356c0 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/296 /content/final_line_segmentation/172_1_1.jpg: 64x640 8 words, 25.2ms
image 2/296 /content/final_line_segmentation/172_1_10.jpg: 64x640 8 words, 21.9ms
image 3/296 /content/final_line_segmentation/172_1_11.jpg: 64x640 11 words, 20.3ms
image 4/296 /content/final_line_segm

In [ ]:
# Sorting Labels of Word detection on the basis of x...
txt_loc = "/content/yolov5/runs/detect/exp3/labels/"
new_sort_label = '/content/sorted_Word_detection/'
flag = 1
sort_detection_label(txt_loc, new_sort_label, flag)

In [ ]:
# Word Segmentation... 
word_labels = "/content/sorted_Word_detection/"
line_images = "/content/final_line_segmentation/"
final_word_dir = "/content/final_word_segmentation/"
os.mkdir(final_word_dir)
word_segmentation(line_images, word_labels)

['172_2_11.jpg', '173_2_21.jpg', '173_2_13.jpg', '172_2_10.jpg', '173_2_10.jpg', '174_1_1.jpg', '173_3_15.jpg', '174_2_9.jpg', '172_4_12.jpg', '172_5_3.jpg', '172_1_18.jpg', '172_3_18.jpg', '173_4_7.jpg', '173_1_6.jpg', '172_3_20.jpg', '172_3_7.jpg', '172_8_17.jpg', '174_4_3.jpg', '173_4_5.jpg', '172_7_10.jpg', '172_4_6.jpg', '172_4_14.jpg', '173_1_15.jpg', '172_8_18.jpg', '174_2_8.jpg', '172_1_13.jpg', '174_3_8.jpg', '173_4_10.jpg', '172_7_9.jpg', '172_3_16.jpg', '173_4_16.jpg', '174_3_10.jpg', '172_3_2.jpg', '172_8_4.jpg', '172_3_13.jpg', '173_4_22.jpg', '172_2_18.jpg', '172_8_12.jpg', '172_2_17.jpg', '172_3_1.jpg', '174_2_11.jpg', '174_4_6.jpg', '174_3_9.jpg', '172_2_7.jpg', '173_2_5.jpg', '172_5_1.jpg', '172_6_2.jpg', '172_7_7.jpg', '174_3_11.jpg', '173_2_15.jpg', '172_1_6.jpg', '172_3_15.jpg', '172_1_7.jpg', '174_1_5.jpg', '173_4_18.jpg', '172_2_12.jpg', '172_6_13.jpg', '172_5_11.jpg', '173_1_7.jpg', '172_1_14.jpg', '173_3_18.jpg', '173_1_17.jpg', '172_5_20.jpg', '173_1_10.jpg', '

In [ ]:
print(dictonary1)
print(dictonary2)
print(dictonary3)

{'/content/files/173/': ['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg'], '/content/files/174/': ['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg'], '/content/files/172/': ['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']}
{'173': ['173_1.jpg', '173_2.jpg', '173_3.jpg', '173_4.jpg', '173_5.jpg'], '174': ['174_1.jpg', '174_2.jpg', '174_3.jpg', '174_4.jpg'], '172': ['172_1.jpg', '172_2.jpg', '172_3.jpg', '172_4.jpg', '172_5.jpg', '172_6.jpg', '172_7.jpg', '172_8.jpg']}
{'173': ['173_1', '173_2', '173_3', '173_4', '173_5'], '174': ['174_1', '174_2', '174_3', '174_4'], '172': ['172_1', '172_2', '172_3', '172_4', '172_5', '172_6', '172_7', '172_8']}


In [ ]:
word_seg_dir = "/content/final_word_segmentation/"
word_seg_list_dir = os.listdir(word_seg_dir)
word_seg_lb_dir = "/content/sorted_Word_detection/"
word_seg_lb_list_dir = os.listdir(word_seg_lb_dir)
print(word_seg_list_dir)
print(word_seg_lb_list_dir)
for key in dictonary3:
  values = dictonary3.get(key)
  for val in values:
    for i in word_seg_list_dir:
      temp = trim_label(i)
      if val == temp:
        from_dir = word_seg_dir + i
        to_dir = "/content/files/" + key + "/words/images/" +val + "/" + i
        if os.path.exists(to_dir) == False:
            shutil.copytree(from_dir, to_dir)
        from_dir_lb = word_seg_lb_dir + i + ".txt"
        to_dir_lb = "/content/files/" + key + "/words/labels/" + i + ".txt"
        if os.path.exists(to_dir_lb) == False:
            shutil.copy(from_dir_lb, to_dir_lb)
        # print(from_dir_lb)
        # print(to_dir_lb)

['173_2_5', '174_2_2', '172_2_16', '173_2_13', '173_1_15', '172_5_1', '173_4_8', '172_3_16', '172_3_20', '173_2_12', '173_3_6', '172_7_13', '172_2_2', '172_6_6', '173_2_3', '172_3_8', '173_1_22', '172_5_17', '173_4_9', '172_5_16', '172_1_16', '173_4_3', '174_2_7', '174_1_1', '172_7_12', '172_3_14', '172_1_7', '172_8_16', '172_8_18', '172_7_1', '172_1_13', '172_3_4', '173_1_14', '172_2_18', '172_6_3', '173_2_1', '172_4_4', '173_1_16', '174_3_2', '172_1_4', '174_2_8', '173_4_2', '172_7_11', '172_8_7', '172_4_16', '174_1_4', '172_1_2', '172_4_11', '172_6_5', '172_3_18', '172_8_10', '172_2_10', '174_2_9', '172_7_2', '172_4_18', '172_5_20', '173_4_12', '174_3_8', '172_2_11', '173_1_23', '174_2_5', '173_1_7', '173_1_13', '172_4_21', '172_8_3', '172_2_3', '172_2_19', '172_5_7', '173_1_12', '173_3_2', '172_1_18', '173_2_15', '173_3_13', '173_4_5', '173_1_17', '172_1_12', '173_3_4', '173_5_4', '174_2_4', '172_7_8', '172_5_4', '172_4_9', '172_3_9', '172_1_11', '172_7_17', '172_3_15', '174_2_10',

# Saving Results

Saving Automatic annotations on the given directory (If want to save the generated outputs: Please uncomment the next code section and assign directory of any folder of your drive to the defined 'final_save' variable).

In [ ]:
# final_save = "/content/drive/MyDrive/thesis/Transition_Outputs/"
# src_dir = "/content/files/"
# final_dir = '/content/automatic_annotation'
# os.rename(src_dir, final_dir)

# temp1 = final_save + "/files/"
# if os.path.exists(temp1) == True:
#   print("The created folder for given image already exist in Drive!")
#   print("Please check in this location: "+final_save)
# else:
#   %cp -rip $final_dir $final_save
#   print("Saved successfully!")

To zip and download the results, comment out and run next two cells. 

In [ ]:
# def zip_results():
#   os.mkdir("/content/test_results")

#   # Zipping the test Results
#   !zip -r /content/test_results/Final_Results.zip /content/automatic_annotation

#   # Downloading the zipped files from our test_results folder
#   # from google.colab import files
#   # files.download("/content/test_results/Final_Results.zip")

# zip_results()

In [ ]:
# files.download('/content/test_results/Final_Results.zip')